In [ ]:
import cv2
from ultralytics import YOLO
import numpy as np
import joblib
from datetime import datetime
import os
from linebot import LineBotApi
from linebot.models import TextSendMessage, ImageSendMessage

# 初始化 LINE Bot
channel_access_token = 'YOUR_CHANNEL_ACCESS_TOKEN' # 替換為您的 Channel Access Token
user_id = 'YOUR_USER_ID' # 替換為您的 User ID
line_bot_api = LineBotApi(channel_access_token)

model = YOLO("yolov8n-pose.pt")
knn_model = joblib.load('knn_model.pkl')
knn_scaler = joblib.load('knn_scaler.pkl')
path = "./fall5.mp4"
cap = cv2.VideoCapture(path)

# 將信心值加入關鍵點座標
def process_image(results):
    r = results[0]
    combined_results = []
    for i in range(len(r.boxes)):
        box_results = []
        for j in range(17):
            x, y = r.keypoints.xyn[i][j].numpy()
            box_results.extend([x, y])
        combined_results.append(box_results)
    return combined_results

# 開始影片偵測
while True:
    ret, frame = cap.read()
    if not ret:
        break
    current_time = datetime.now()
    # Perform pose detection
    results = model(frame, conf=0.3)  # 偵測影片中人體關節點
    annotated_frame = results[0].plot()  # 將關鍵點畫到影片上
    r = results[0]  # 輸出結果
    # 若有偵測到人
    if len(results[0].boxes) >= 1:
        data = np.array(process_image(results))  # 將資料轉成2D numpy array
        data = knn_scaler.transform(data)
        y_pred = knn_model.predict(data)
        print(y_pred)
        if y_pred[0] == 2:
            fall_current_time = current_time
            folder_name = 'Fall_img'
            os.makedirs(folder_name, exist_ok=True)
            time_text = fall_current_time.strftime("%Y-%m-%d %H:%M:%S")
            file_path = os.path.join(folder_name, fall_current_time.strftime("%Y-%m-%d_%H%M%S_fall.jpg"))
            cv2.imwrite(file_path, frame)
            print("output save")
            
            # 通知 LINE，並將圖片送出
            try:
                line_bot_api.push_message(user_id, TextSendMessage(text=f'Person Fall Detected at {time_text}'))
                image_url = f'file://{file_path}'
                line_bot_api.push_message(user_id, ImageSendMessage(original_content_url=image_url, preview_image_url=image_url))
            except Exception as e:
                print(f'Error sending message: {e}')
    
    cv2.imshow('YOLOv8 Pose Detection', annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
